In [ ]:
# Define the grammar
grammar = {
    "S'": ["E"],
    "E": ["T - E", "T"],
    "T": ["F x T", "F"],
    "F": ["id"]
}

# Compute the LR(1) items for a grammar
def lr1_items(grammar):
    start = grammar.start()
    productions = grammar.productions()
    items = []
    for i, prod in enumerate(productions):
        # Add item [A -> .alpha, a] for each production
        # A -> alpha in the grammar
        A = prod.lhs()
        alpha = prod.rhs()
        dot = 0
        lookahead = set()
        item = (i, dot, frozenset(lookahead))
        items.append((A, item))
        for j, symbol in enumerate(alpha):
            dot = j + 1
            lookahead = set()
            if dot < len(alpha):
                # Compute lookahead for item [A -> alpha, .B, b]
                b = alpha[dot]
                next_symbols = set(alpha[dot+1:])
                lookahead |= first(grammar, next_symbols, set())
                if nullable(grammar, next_symbols):
                    lookahead |= item[2]
            item = (i, dot, frozenset(lookahead))
            items.append((symbol, item))
        # Add item [A -> alpha., a] for each production
        # A -> alpha in the grammar
        if not alpha:
            dot = 1
            lookahead = set()
            item = (i, dot, frozenset(lookahead))
            items.append((A, item))
    return items

def lr1_parse_table(lr1_items):
    # Compute the set of non-terminal and terminal symbols
    symbols = set()
    for items in lr1_items:
        for item in items:
            symbols.add(str(item.symbol()))
            symbols |= set(symbol.symbol() for symbol in item.productions())
            symbols |= set(str(symbol) for symbol in item[2])
    symbols -= set([''])

    # Initialize the parsing table
    table = {}
    for i, items in enumerate(lr1_items):
        for item in items:
            A, alpha, beta, a = item
            if beta:
                j = next(j for j, items in enumerate(lr1_items) if (A, alpha + [beta[0]], beta[1:], a) in items)
                if beta[0] not in table.get((i, beta[0]), {}):
                    table[(i, beta[0])] = ('s', j)
                else:
                    raise ValueError('LR(1) conflict')
            else:
                for a in item[2]:
                    if (i, a) not in table:
                        table[(i, a)] = ('r', A, len(alpha))
                    else:
                        raise ValueError('LR(1) conflict')
                if (i, '$') not in table:
                    table[(i, '$')] = ('a',)
                else:
                    raise ValueError('LR(1) conflict')
    return table


# Parse a string using the LR(1) parser
# Parse a string using the LR(1) parser
def lr1_parse(string, parse_table):
    stack = [0]
    i = 0
    output = []
    while True:
        state = stack[-1]
        lookahead = string[i] if i < len(string) else '$'
        action = parse_table.get((state, lookahead))
        if not action:
            raise ValueError('Invalid input')
        op, *args = action
        if op == 's':
            stack.append(args[0])
            stack.append(lookahead)
            i += 1
            output.append(('shift', lookahead))
        elif op == 'r':
            A, length = args
            for _ in range(2 * length):
                stack.pop()
            state = stack[-1]
            stack.append(A)
            if (state, A) not in parse_table:
                raise ValueError('Invalid input')
            stack.append(parse_table[(state, A)][1])
            output.append(('reduce', A, length))
        elif op == 'a':
            output.append(('accept',))
            break
        else:
            raise ValueError('Invalid action')
        print(stack, string[i:], output)
    return output






In [ ]:
def nullable(grammar, symbols):
    """
    Returns True if the given symbols can derive the empty string, False otherwise.
    """
    if not symbols:
        return True
    for symbol in symbols:
        if isinstance(symbol, str) or not nullable(grammar, grammar.productions(lhs=symbol)):
            return False
    return True


def first(grammar, symbols, seen=set()):
    first_set = set()
    terminals = {symbol for prod in grammar.productions() for symbol in prod.rhs() if isinstance(symbol, str)}
    for symbol in symbols:
        if symbol in terminals:
            first_set.add(symbol)
        elif isinstance(symbol, nltk.grammar.Nonterminal) and symbol not in seen:
            seen.add(symbol)
            productions = grammar.productions(lhs=symbol)
            for production in productions:
                first_set |= first(grammar, production.rhs(), seen)
    return first_set


In [ ]:
import nltk

# Create a Nonterminal object for 'E'
E = nltk.grammar.Nonterminal('E')

# Create a dictionary representing the grammar
grammar_dict = {
    E: [[E, '-', E], [E]],
    'T': [['F', '*', 'T'], ['F']],
    'F': [['id']]
}

# Create a list of production rules
rules = []
for lhs, rhss in grammar_dict.items():
    for rhs in rhss:
        rules.append(nltk.grammar.Production(lhs, rhs))

# Create a CFG object from the list of production rules
grammar = nltk.grammar.CFG(E, rules)
lr1_items = lr1_items(grammar)

# Generate the LR(1) parsing table from the LR(1) items
parse_table = lr1_parse_table(lr1_items)

# Parse a string using the LR(1) parser
string = "a b c d"
lr1_parse(string, parse_table)
